In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df1 = pd.read_excel('/content/drive/MyDrive/Exp_Proj_5490/Project/EP5490_Data.xlsx')

In [ ]:
df = df1.copy()

In [ ]:
df

,Annonymized ID,Job Code,Job Title,Job Function,Job Category,Job Group,Compa Ratio,Pay Level,Work Location,Work Country,...,Termination Date,Tenure,Tenure Bucket,Base Pay Mid Point Annualized,Currency Conversion Rate,Generation,Work Structure,Termination Type,Termination Reason,Cost to Replace Employee Multiplier
0,1111,1345_USA BA,Principal Engineer,Engineering,Engineering Professional,Professional,0.675,P6,Cupertino United States,United States,...,2017-09-25,32.194521,20+ Years,270004.8,1.0,Boomer,NaN,Involuntary Termination,Position Elimination / RIF Involuntary,1.25
1,1112,8804_USA BA,Managing Principal Engineer,Engineering,Management,Management,0.979,M4,Fremont United States,United States,...,2022-02-04,31.079452,20+ Years,203008.0,1.0,Boomer,NaN,Voluntary Termination,Compensation/Pay,1.25
2,1113,1600_USA NTL,Staff Engineer,Engineering,Engineering Professional,Professional,1.146,P4,Remote US,United States,...,NaT,33.101370,20+ Years,118996.8,1.0,Boomer,Remote,NaN,NaN,1.25
3,1115,3627_USA BA,Sr Staff Program/Project Manager,Information Technology,Professional,Professional,0.941,P5,Fremont United States,United States,...,NaT,32.969863,20+ Years,184995.2,1.0,Gen X,Onsite,NaN,NaN,1.25
4,1116,2692_USA NTL,Technologist,Engineering,Engineering Professional,Professional,0.932,P7,Longmont United States,United States,...,NaT,32.947945,20+ Years,249995.2,1.0,Boomer,Onsite,NaN,NaN,1.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25990,87558,8809_USA NTL,Managing Principal Engineer,Engineering,Management,Management,1.043,M4,Longmont United States,United States,...,2022-12-07,32.213699,20+ Years,184995.2,1.0,Boomer,Onsite,Involuntary Termination,Position Elimination / RIF Involuntary,1.25
25991,87559,2042_USA BA,Sr Engineering Director,Engineering,Management,Management,0.841,M6,Fremont United States,United States,...,NaT,33.421918,20+ Years,276993.6,1.0,Gen X,Onsite,NaN,NaN,1.25
25992,87560,1343_USA NTL,Principal Engineer,Engineering,Engineering Professional,Professional,0.767,P6,Longmont United States,United States,...,2018-10-12,28.002740,20+ Years,243006.4,1.0,Boomer,NaN,Other Termination,Retirement,1.25
25993,87562,1558_USA NTL,Sr Staff Engineer,Engineering,Engineering Professional,Professional,0.893,P5,Longmont United States,United States,...,2018-10-12,27.947945,20+ Years,167003.2,1.0,Boomer,NaN,Other Termination,Retirement,1.25


***DATA CLEANING AND INSPECTING***

**** Column Name Editing ****

In [ ]:
df.columns = df.columns.str.lower()

In [ ]:
df = df.rename(columns={'annonymized id': 'anon id', 'termination reason ': 'termination reason'})

*** Dropping NA's ***

In [ ]:
df = df.dropna(subset=['generation', 'pay level', 'currency conversion rate']).drop_duplicates()

*** Editing Work Country / Location / Structure for clarity ***

In [ ]:
# removing work country from work location, given we have a work country column already
df['work location'] = df.apply(lambda row: row['work location'].replace(row['work country'], '').strip(), axis=1)
df = df.rename(columns = {'work location':'work city'})

In [ ]:
# editing work city names
df['work city'] = df['work city'].replace({'Remote  - British Columbia': 'British Columbia',
                                         'Portland, Oregon': 'Portland',
                                         'Remote Republic of': 'Ireland',
                                         'Remote  - Ontario': 'Ontario',
                                         'Remote Northern Ireland': 'N Ireland',
                                          '': 'Hong Kong'})


*** Tenure Rounding ***

In [ ]:
df['tenure'] = df['tenure'].round(2)

*** Feature Engineering ***

In [ ]:
# feature engineering - cost to replace employee
df['cost to replace employee'] = df['base pay mid point annualized'] * df['cost to replace employee multiplier']

In [ ]:
# currency conversion
df['usd base pay mid point'] = df['base pay mid point annualized'] * df['currency conversion rate']
df = df.drop(['base pay mid point annualized', 'currency conversion rate'], axis=1)

*** Sanity Checking Compa Ratio / dropping excessive values ***

In [ ]:
df = df[df['compa ratio'] <= 2]
df = df[df['compa ratio'] >= .25]

In [ ]:
# removing the 6 mystery nulls in work city
df = df.dropna(subset = ['work city'])

In [ ]:
# filling work structure NA's
df['work structure'] = df['work structure'].fillna('Onsite')

In [ ]:
# keeping only active employees
df = df.drop(df[(df['termination type'] == 'Involuntary Termination') |
 (df['termination type'] == 'Other Termination') |
  (df['termination type'] == 'Release Termination') |
   (df['termination type'] == 'No Show') |
   (df['termination type'] == 'Voluntary Termination')].index)


In [ ]:
bins = [0, 0.25, 0.5, 0.75, 1, float('inf')]

# Define the bin labels
labels = ['1-25%', '25-50%', '50-75%', '75-100%', '100%+']

# Bin the 'compensation ratio' column
df['compa bucket'] = pd.cut(df['compa ratio'], bins=bins, labels=labels, right=False)


In [ ]:
# Define the bin edges for tenure
tenure_bins = [0, 1, 5, 10, 20, float('inf')]

# Define the bin labels for tenure
tenure_labels = ['<1', '1-5', '5-10', '10-20', '20+']

# Custom function to map and bin the 'tenure bucket' column
def map_and_bin_tenure(bucket):
    if bucket == '< 1 Year':
        return '0'
    elif bucket == '1 - 3 Years':
        return '3'
    elif bucket == '3 - 5 Years':
        return '2'
    elif bucket == '5 - 10 Years':
        return '3'
    elif bucket == '10 - 20 Years':
        return '4'
    elif bucket == '20+ Years':
        return '5'
    else:
        return None

# Apply the custom function to map and bin the 'tenure bucket' column
df['tenure bucket'] = df['tenure bucket'].apply(map_and_bin_tenure)

In [ ]:
df['tenure bucket'] = df['tenure bucket'].astype(str).astype(int)

In [ ]:
# df['termination date'] = pd.to_datetime(df['termination date'])
# df['quarter'] = df['termination date'].dt.year.astype(str) + ' Q' + df['termination date'].dt.quarter.astype(str)

In [ ]:
# # Create a list of years from 2016 to 2024
# years = range(2016, 2025)

# # Create an empty DataFrame to store the results
# df_quarters = pd.DataFrame()

# # Loop through each year and quarter
# for year in years:
#     for quarter in range(1, 5):  # Quarters range from 1 to 4
#         quarter_name = f"{year} Q{quarter}"
#         df_quarters[quarter_name] = np.where(df['quarter'] <= f"{year}-Q{quarter}", "Active", "NaN")

# # Concatenate the new DataFrame with df
# df = pd.concat([df, df_quarters], axis=1)

In [ ]:
df = df.drop(columns = ['job code', 'employee status', 'termination date',
      'termination reason', 'termination type','cost to replace employee multiplier', 'cost to replace employee',
       'usd base pay mid point', 'compa ratio', # created compa bucket

In [ ]:
df.shape

(11668, 14)

In [ ]:
df = df.reset_index(drop=True)

*** EXPORT CLEAN DATA ***

In [ ]:
df.to_csv('active_clean_df_copy.csv', index=False)